***GENERATED CODE FOR alzimerssprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from os import listdir
import pyspark.sql.functions as F
from pyspark.sql.functions import when, col, regexp_replace
from pyspark.sql.types import StringType, DoubleType, IntegerType, TimestampType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Argentina', 'max': 'USA', 'missing': '0', 'distinct': '20'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Physical Activity Level', 'transformation_label': 'String Indexer'}], 'feature': 'Physical Activity Level', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Physical Activity Level'}, {'feature_label': 'Physical Activity Level', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Physical Activity Level')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Smoking Status', 'transformation_label': 'String Indexer'}], 'feature': 'Smoking Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Current', 'max': 'Never', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Smoking Status'}, {'feature_label': 'Smoking Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Smoking Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Alcohol Consumption', 'transformation_label': 'String Indexer'}], 'feature': 'Alcohol Consumption', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Never', 'max': 'Regularly', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Alcohol Consumption'}, {'feature_label': 'Alcohol Consumption', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Alcohol Consumption')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Diabetes', 'transformation_label': 'String Indexer'}], 'feature': 'Diabetes', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Diabetes'}, {'feature_label': 'Diabetes', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Diabetes')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Hypertension', 'transformation_label': 'String Indexer'}], 'feature': 'Hypertension', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Hypertension'}, {'feature_label': 'Hypertension', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Hypertension')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Cholesterol Level', 'transformation_label': 'String Indexer'}], 'feature': 'Cholesterol Level', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Normal', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Cholesterol Level'}, {'feature_label': 'Cholesterol Level', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Cholesterol Level')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': "Family History of Alzheimer's", 'transformation_label': 'String Indexer'}], 'feature': "Family History of Alzheimer's", 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Family History of Alzheim...'}, {'feature_label': "Family History of Alzheimer's", 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Family History of Alzheimer's')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Depression Level', 'transformation_label': 'String Indexer'}], 'feature': 'Depression Level', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Depression Level'}, {'feature_label': 'Depression Level', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Depression Level')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sleep Quality', 'transformation_label': 'String Indexer'}], 'feature': 'Sleep Quality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Average', 'max': 'Poor', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sleep Quality'}, {'feature_label': 'Sleep Quality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sleep Quality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dietary Habits', 'transformation_label': 'String Indexer'}], 'feature': 'Dietary Habits', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Average', 'max': 'Unhealthy', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Dietary Habits'}, {'feature_label': 'Dietary Habits', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dietary Habits')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Air Pollution Exposure', 'transformation_label': 'String Indexer'}], 'feature': 'Air Pollution Exposure', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Air Pollution Exposure'}, {'feature_label': 'Air Pollution Exposure', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Air Pollution Exposure')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Employment Status', 'transformation_label': 'String Indexer'}], 'feature': 'Employment Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Employed', 'max': 'Unemployed', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Employment Status'}, {'feature_label': 'Employment Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Employment Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Marital Status', 'transformation_label': 'String Indexer'}], 'feature': 'Marital Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Married', 'max': 'Widowed', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Marital Status'}, {'feature_label': 'Marital Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Marital Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Genetic Risk Factor', 'transformation_label': 'String Indexer'}], 'feature': 'Genetic Risk Factor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Genetic Risk Factor'}, {'feature_label': 'Genetic Risk Factor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Genetic Risk Factor')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Social Engagement Level', 'transformation_label': 'String Indexer'}], 'feature': 'Social Engagement Level', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Social Engagement Level'}, {'feature_label': 'Social Engagement Level', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Social Engagement Level')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Income Level', 'transformation_label': 'String Indexer'}], 'feature': 'Income Level', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Income Level'}, {'feature_label': 'Income Level', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Income Level')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Stress Levels', 'transformation_label': 'String Indexer'}], 'feature': 'Stress Levels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Stress Levels'}, {'feature_label': 'Stress Levels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Stress Levels')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Urban vs Rural Living', 'transformation_label': 'String Indexer'}], 'feature': 'Urban vs Rural Living', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Rural', 'max': 'Urban', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Urban vs Rural Living'}, {'feature_label': 'Urban vs Rural Living', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Urban vs Rural Living')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Alzheimers_Diagnosis', 'transformation_label': 'String Indexer'}], 'feature': 'Alzheimers_Diagnosis', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Alzheimers_Diagnosis'}, {'feature_label': 'Alzheimers_Diagnosis', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Alzheimers_Diagnosis')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run alzimersspredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	alzheimerspredictiondataset = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/alzheimers_prediction_dataset (1).csv', 'filename': 'alzheimers_prediction_dataset (1).csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(alzheimerspredictiondataset)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run alzimersspredictionHooks.ipynb
try:
	#transformationPreExecutionHook()

	alzimersspredictionautofe = TransformationMain.run(alzheimerspredictiondataset,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Argentina", "max": "USA", "missing": "0", "distinct": "20"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "70.46", "stddev": "12.8", "min": "50", "max": "94", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Education Level", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.52", "stddev": "5.82", "min": "0", "max": "19", "missing": "0"}, "updatedLabel": "Education Level"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BMI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "26.51", "stddev": "4.77", "min": "18.5", "max": "35.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BMI"}, {"transformationsData": [{"feature_label": "Physical Activity Level", "transformation_label": "String Indexer"}], "feature": "Physical Activity Level", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Physical Activity Level"}, {"transformationsData": [{"feature_label": "Smoking Status", "transformation_label": "String Indexer"}], "feature": "Smoking Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Current", "max": "Never", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Smoking Status"}, {"transformationsData": [{"feature_label": "Alcohol Consumption", "transformation_label": "String Indexer"}], "feature": "Alcohol Consumption", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Never", "max": "Regularly", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Alcohol Consumption"}, {"transformationsData": [{"feature_label": "Diabetes", "transformation_label": "String Indexer"}], "feature": "Diabetes", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Diabetes"}, {"transformationsData": [{"feature_label": "Hypertension", "transformation_label": "String Indexer"}], "feature": "Hypertension", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Hypertension"}, {"transformationsData": [{"feature_label": "Cholesterol Level", "transformation_label": "String Indexer"}], "feature": "Cholesterol Level", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Normal", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Cholesterol Level"}, {"transformationsData": [{"feature_label": "Family History of Alzheimer's", "transformation_label": "String Indexer"}], "feature": "Family History of Alzheimer's", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Family History of Alzheim..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Cognitive Test Score", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "64.61", "stddev": "20.05", "min": "30", "max": "99", "missing": "0"}, "updatedLabel": "Cognitive Test Score"}, {"transformationsData": [{"feature_label": "Depression Level", "transformation_label": "String Indexer"}], "feature": "Depression Level", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Depression Level"}, {"transformationsData": [{"feature_label": "Sleep Quality", "transformation_label": "String Indexer"}], "feature": "Sleep Quality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Average", "max": "Poor", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sleep Quality"}, {"transformationsData": [{"feature_label": "Dietary Habits", "transformation_label": "String Indexer"}], "feature": "Dietary Habits", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Average", "max": "Unhealthy", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Dietary Habits"}, {"transformationsData": [{"feature_label": "Air Pollution Exposure", "transformation_label": "String Indexer"}], "feature": "Air Pollution Exposure", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Air Pollution Exposure"}, {"transformationsData": [{"feature_label": "Employment Status", "transformation_label": "String Indexer"}], "feature": "Employment Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Employed", "max": "Unemployed", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Employment Status"}, {"transformationsData": [{"feature_label": "Marital Status", "transformation_label": "String Indexer"}], "feature": "Marital Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Married", "max": "Widowed", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Marital Status"}, {"transformationsData": [{"feature_label": "Genetic Risk Factor", "transformation_label": "String Indexer"}], "feature": "Genetic Risk Factor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Genetic Risk Factor"}, {"transformationsData": [{"feature_label": "Social Engagement Level", "transformation_label": "String Indexer"}], "feature": "Social Engagement Level", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Social Engagement Level"}, {"transformationsData": [{"feature_label": "Income Level", "transformation_label": "String Indexer"}], "feature": "Income Level", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Income Level"}, {"transformationsData": [{"feature_label": "Stress Levels", "transformation_label": "String Indexer"}], "feature": "Stress Levels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Stress Levels"}, {"transformationsData": [{"feature_label": "Urban vs Rural Living", "transformation_label": "String Indexer"}], "feature": "Urban vs Rural Living", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Rural", "max": "Urban", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Urban vs Rural Living"}, {"transformationsData": [{"feature_label": "Alzheimers_Diagnosis", "transformation_label": "String Indexer"}], "feature": "Alzheimers_Diagnosis", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Alzheimers_Diagnosis"}]}))

	#transformationPostExecutionHook(alzimersspredictionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run alzimersspredictionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(alzimersspredictionautofe, ["Age", "Education Level", "BMI", "Cognitive Test Score", "Country_stringindexer", "Gender_stringindexer", "Physical Activity Level_stringindexer", "Smoking Status_stringindexer", "Alcohol Consumption_stringindexer", "Diabetes_stringindexer", "Hypertension_stringindexer", "Cholesterol Level_stringindexer", "Family History of Alzheimer's_stringindexer", "Depression Level_stringindexer", "Sleep Quality_stringindexer", "Dietary Habits_stringindexer", "Air Pollution Exposure_stringindexer", "Employment Status_stringindexer", "Marital Status_stringindexer", "Genetic Risk Factor_stringindexer", "Social Engagement Level_stringindexer", "Income Level_stringindexer", "Stress Levels_stringindexer", "Urban vs Rural Living_stringindexer"], "Alzheimers_Diagnosis_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

